In [7]:
%matplotlib inline
import re
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib as mpl
from matplotlib.colors import SymLogNorm
import scienceplots

import astropy.units as u
from astropy import constants
from astropy.table import Table
from astropy.coordinates import SkyCoord,FK5,ICRS
from astropy.io import fits,ascii
from astropy.wcs import WCS, utils as wcsutils
from astropy.stats import sigma_clipped_stats
import astropy.visualization as vis
from astropy.modeling import models, fitting
from astropy.nddata import StdDevUncertainty, Cutout2D
from astropy.convolution import Gaussian2DKernel,Gaussian1DKernel
from astropy.cosmology import Planck18
from scipy.signal import find_peaks
#from scipy.constants import c



from photutils.aperture import EllipticalAperture, SkyEllipticalAperture

import os, sys, glob, pdb, scipy

from importlib import reload
import pyregion as pyreg
from spectral_cube import SpectralCube
from reproject import reproject_interp
import regions
from regions.shapes.circle import CircleSkyRegion,CirclePixelRegion
from regions import CircleAnnulusSkyRegion, CircleAnnulusPixelRegion, RectangleSkyRegion
from regions import PixCoord
sys.path.append('/disk/bifrost/yuanze/KBSS/MUSEQSO')
import run_cubetools_MUSE as ctools
import makeMask_MUSE as mmask
reload(ctools)
reload(mmask)
sys.path.append('/disk/bifrost/yuanze/software/GalfitS/src')
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
import images as IM
reload(IM)

brightu = r"Brightness $\rm (10^{-4}~erg~s^{-1}~arcsec^{-2}~cm^{-2}~\AA^{-1})$"
def imshow_astro(img, wcsinfo = None, figsize = (10,10), colorbar =True,
               cblabel="", cbfrac = 0.035, norm = None,
               stretch = vis.LinearStretch(), cmap = "hot",
               vrange = (None, None)):
    
    _, med, std = sigma_clipped_stats(img.data)
    
    fig =  plt.figure(figsize = figsize)
    if wcsinfo:
        ax = plt.subplot(projection = wcsinfo)
    else:
        ax = plt.subplot()
    
    vmin, vmax = vrange
    
    if not vmin:
        vmin = med
    if not vmax:
        vmax = med + 10*std
    if not norm:
        norm = vis.ImageNormalize(vmin = vmin, vmax = vmax, stretch = stretch)
    im = ax.imshow(img, norm = norm, cmap = cmap)
    if colorbar:
        cb = plt.colorbar(im, label = cblabel, fraction =cbfrac)
    return fig, ax, im
KBSSpath="/disk/bifrost/yuanze/KBSS"
qsos_bright = ascii.read(KBSSpath+"/KCWI/qsos_bright.kcwi",format="ipac")

#MUSEQSO= ascii.read(KBSSpath+"/KCWI/MUSEQSO_machine_readable_updated2.list",format="ipac")
#museqso_path = os.path.join(KBSSpath, "MUSEQSO")
#votable_file = os.path.join("fixed_file.xml")

In [11]:
qsos_bright

Name,Field,RA,Dec,x,y,EBV,z_simbad,z_sys,zerr,contam,u,g,r,i,z,MB,Comments
unit,unit,unit,unit,unit,unit,unit,unit,unit,unit,unit,unit,unit,unit,unit,unit,unit,unit
str11,str5,float64,float64,float64,float64,float64,float64,float64,float64,str5,float64,float64,float64,float64,float64,float64,str71
Q0100+13,Q0100,15.797,13.272,50.46,50.41,0.031,2.686,2.721,0.001,False,17.4,16.7,16.5,16.5,--,-26.4472,--
HS0105+1619,Q0105,17.027,16.597,52.31,53.45,0.058,2.64,2.652,0.001,False,18.1,17.19,17.0,16.87,16.67,--,--
Q0142-10,Q0142,26.319,-9.755,55.86,54.4,0.028,2.7312,2.743,0.001,True,17.54,16.85,16.73,16.64,16.6,-26.1957,lensed QSO
Q0207-003,Q0207,32.461,-0.085,49.95,50.93,0.024,2.85,2.872,0.001,False,18.08,17.02,16.92,16.84,16.76,--,--
Q0821+3107,Q0821,125.282,31.131,54.68,52.86,0.039,2.624,2.616,0.001,False,18.08,17.26,17.04,16.87,16.66,--,--
Q1009+29,Q1009,152.982,29.695,48.79,48.47,0.023,2.62,2.652,0.001,False,16.73,16.16,16.09,16.02,15.89,-26.7954,--
Q1217+490,Q1217,184.879,49.681,47.9,43.83,0.015,2.698,2.704,0.001,False,17.77,17.09,16.92,16.79,16.7,-25.9884,--
Q1442+2931,Q1442,221.224,29.318,51.0,45.68,0.015,2.638,2.66,0.001,True,17.31,16.41,16.41,16.33,16.09,--,"K=14.07,J=15.02"


In [95]:

#reload(mmask)
from datetime import datetime# Making white-light cutout
root_directory = KBSSpath
condition=[qsos_bright['Field']!="Q1623",qsos_bright['Field']!="Q0142"]
source_table = qsos_bright[(qsos_bright['Field']!="Q1623") & (qsos_bright['Field']!="Q0142")]

#print("Number of directories found:",len(all_directories))
#gc.set_debug(gc.DEBUG_LEAK)
radius=6
dovar=False
for element in source_table:
    print(element)
    qname = element["Name"]
    field = element["Field"]
    objname="qso"
    subdapath=KBSSpath+"/"+field+"/"+objname.upper()
    #CubRunpath=KBSSpath+"/CubEx_run/"+field
    fn=subdapath+"/{}-{}_icubes_wcs.PSFSub.fits".format(field.lower(),objname,int(radius/0.3))
    hdu0=fits.open(fn)
    hdr = hdu0[0].header
    wc = (np.arange(hdr['NAXIS3']) - hdr['CRPIX3'] + 1) * hdr['CD3_3'] + hdr['CRVAL3']
    #NSubfile = adp_prefix+".PSFSub.fits"
    #zmask = np.loadtxt(subdir+"/zmask.txt").astype(np.int64)
    zmask = mmask.determine_mask(wc,"continuum", element['z_sys'])
    xpix = element["x"]
    ypix = element["y"]
    wimage = np.nansum(hdu0[0].data[zmask[0]:zmask[1],:],axis=0)
    wimage[wimage==0] = np.nan
    header = fits.Header()
    header['DATE'] = datetime.now().strftime('%Y-%m-%dT%H:%M:%S')
    header['CREATOR'] = 'Yuanze Ding'
    header['COMMENT'] = 'White-light cutout from 3D data cube'
    header['YSOURCE'] = ypix
    header['XSOURCE'] = xpix
    header['OBJECT'] = qname
    # Extract WCS information from the original header
    wcs_keys = ['CTYPE1', 'CTYPE2', 'CRVAL1', 'CRVAL2', 'CRPIX1', 'CRPIX2', 'CDELT1', 'CDELT2', 'CUNIT1', 'CUNIT2', 'CD1_1', 'CD1_2', 'CD2_1', 'CD2_2']
    for key in wcs_keys:
        if key in hdr:
            header[key] = hdr[key]

    hdu = fits.PrimaryHDU(wimage,header=header)
    output_fn = subdapath+"/{}-{}_icubes_wcs_{}.PSFSub.white.fits".format(field.lower(),objname,int(radius/0.3))
    hdulist = fits.HDUList([hdu])
    hdulist.writeto(output_fn, overwrite=True)
    if dovar:
        fnvar=subdapath+"/{}-{}_vcubes.fits".format(field.lower(),objname)
        hdu0_var=fits.open(fnvar)
        hduvar = fits.PrimaryHDU(wvar,header=header)
        wvar = np.nansum(hdu0_var[0].data[zmask[0]:zmask[1],:],axis=0)
        wvar[wvar==0] = np.nan
        output_fnvar = subdapath+"/{}-{}_icubes_wcs.whitevar.fits".format(field.lower(),objname)
        hdulist = fits.HDUList([hduvar])
        hdulist.writeto(output_fnvar, overwrite=True)
#hdu2=fits.open(KBSSpath+"/MUSEQSO/Q-0347-383/ADP.2016-06-02T13:21:57.103.fits")

  Name   Field   RA    Dec     x     y    EBV  z_simbad z_sys  zerr contam  u    g    r    i    z      MB    Comments
  unit    unit  unit   unit   unit  unit  unit   unit    unit  unit  unit  unit unit unit unit unit   unit     unit  
-------- ----- ------ ------ ----- ----- ----- -------- ----- ----- ------ ---- ---- ---- ---- ---- -------- --------
Q0100+13 Q0100 15.797 13.272 50.46 50.41 0.031    2.686 2.721 0.001  False 17.4 16.7 16.5 16.5   -- -26.4472       --
    Name    Field   RA    Dec     x     y    EBV  z_simbad z_sys  zerr contam  u     g    r     i     z    MB  Comments
    unit     unit  unit   unit   unit  unit  unit   unit    unit  unit  unit  unit  unit unit  unit  unit unit   unit  
----------- ----- ------ ------ ----- ----- ----- -------- ----- ----- ------ ---- ----- ---- ----- ----- ---- --------
HS0105+1619 Q0105 17.027 16.597 52.31 53.45 0.058     2.64 2.652 0.001  False 18.1 17.19 17.0 16.87 16.67   --       --
   Name   Field   RA    Dec     x     y    EBV  

In [99]:
import warnings
from regions import Regions
reload(ctools)
root_directory = KBSSpath
condition=[qsos_bright['Field']!="Q1623",qsos_bright['Field']!="Q0142"]
source_table = qsos_bright[(qsos_bright['Field']!="Q1623") & (qsos_bright['Field']!="Q0142")]

print("Number of directories found:",len(source_table))
#gc.set_debug(gc.DEBUG_LEAK)
radius=6
cutr=20

for element in source_table:
    qname = element["Name"]
    field = element["Field"]
    print(qname)
    
    objname="qso"
    subdapath=KBSSpath+"/"+field+"/"+objname.upper()
    CubRunpath=KBSSpath+"/CubEx_run/"+field
    ra = element['RA']
    dec = element['Dec']
    sc= SkyCoord(ra,dec,unit=(u.deg,u.deg))
   # qname="LBQS 2139-4434"
    fname=subdapath+"/{}-{}_icubes_wcs_{}.PSFSub.white.fits".format(field.lower(),objname,int(radius/0.3))
    fnamevar=subdapath+"/{}-{}_icubes_wcs.whitevar.fits".format(field.lower(),objname,int(radius/0.3))
    hduvar=fits.open(fnamevar)
    var=hduvar[0].data
    #var[var==0]=np.nan
    mean_sky_var, _, _ = sigma_clipped_stats(var, sigma=3.0, maxiters=None)
    img = IM.image(fname,hdu=0,unit=u.Jy)
    xpix = element["x"]
    ypix = element["y"]
    wcs=WCS(img.data.header)
    
    
    reg= ctools.read_regions(KBSSpath+f"/AbsHosts_nLOS_reg/{field}_AbsHost-nLOS.reg",pix_scale=wcs.pixel_scale_matrix[1,1]*3600)
    artists=[]
    metas=[]
    for r in reg:
        artists.append(r.to_pixel(wcs).as_artist())
        meta=dict(r.meta)
        meta["fontsize"]=5
        metas.append(meta)
    center = [ypix,xpix]
    imcut, cp = img.img_cut(sc.ra.value,sc.dec.value,cutr)
    sky_mean, sky_median, sky_std = sigma_clipped_stats(imcut, sigma=3.0, maxiters=None)
    print("sky level from vcube:",np.sqrt(mean_sky_var),"; from sigma clipping:",sky_std,"; bkg level:",sky_mean)
    img.sources_skycord_cut = [xpix,ypix]
    #img.cut_mask_image = np.zeros_like(img.cut_sigma_image)
    sdmask = ctools.create_circular_mask(img.data.shape, center, 7)
    try:
        with warnings.catch_warnings(record=True) as w:
            warnings.simplefilter("always")
            img.cut_mask_image = img.generate_cutmask(4,nsigma=2,sky_level=np.sqrt(mean_sky_var),sdmask=sdmask,data = img.data.data,deblend=False,addgrow=1,nlevels=32,source_dia=5,contrast=0.001)
    except:
        for warning in w:
            print(f"Warning: {warning.message}")
        continue
    ctools.maskplot(img.data.data,img.cut_mask_image,meta=metas,artist=artists,output=root_directory+"/diagnostic/"+qname+"_maskmap.pdf",center=center)
    outname = subdapath+"/{}-{}_icubes_wcs_{}.PSFSub.mask.fits".format(field.lower(),objname,int(radius/0.3))
    ctools.write_fits_cube(img.cut_mask_image.astype(np.int32),img.data.header,outname)
    #NSubfile = adp_prefix+".PSFSub.fits"
    #zmask = np.loadtxt(subdir+"/zmask.txt",dtype=int)

Number of directories found: 12
Q0100+13
15.797 13.272
50.87291671204359 53.04720000122385
sky level from vcube: 0.060331028301538046 ; from sigma clipping: 0.061062913 ; bkg level: 0.0076338905


HS0105+1619
17.027 16.597
49.43423838716613 50.51880152295992
sky level from vcube: 0.04183433563785241 ; from sigma clipping: 0.046049867 ; bkg level: 0.013854104


Q0207-003
32.461 -0.085
53.584 52.5570000000555
sky level from vcube: 0.04611689432395581 ; from sigma clipping: 0.049680743 ; bkg level: 0.010127598


Q0821+3107
125.282 31.131
52.2275131399579 53.38760347937567
sky level from vcube: 0.06266494291454598 ; from sigma clipping: 0.06549078 ; bkg level: 0.018385166


Q1009+29
152.982 29.695
44.94308411834539 49.088006169389686
sky level from vcube: 0.05592064729346742 ; from sigma clipping: 0.062023677 ; bkg level: 0.010670031


...

Q1549+1919
237.969 19.185
44.83325777287522 75.20000398181105
sky level from vcube: 0.0553179392147253 ; from sigma clipping: 0.06644044 ; bkg level: 0.016030656


HS1603+3820
241.231 38.201
38.58547702042855 53.01600114513727
sky level from vcube: 0.07311087287027777 ; from sigma clipping: 0.05997777 ; bkg level: 0.0190582


HS1700+64
255.252 64.203
49.83666092431496 54.988006240866376
sky level from vcube: 0.05994306360593578 ; from sigma clipping: 0.058647398 ; bkg level: 0.014084321


Q2206-199
332.217 -19.733
51.248187406707295 56.767999946570555
sky level from vcube: 0.07573714982155874 ; from sigma clipping: 0.07861409 ; bkg level: 0.014603184


Q2343+12
356.618 12.816
45.03452777960135 49.60000305773727
sky level from vcube: 0.06013925856539714 ; from sigma clipping: 0.063121475 ; bkg level: 0.016699161


{'label': 'Q0100  2.721'}

In [81]:
artists[0].center

(50.04648997238098, 49.072736870607734)

In [67]:
reg[0].meta

{'label': 'Q0100  2.721'}

In [71]:
plt.add_artist(artists[0])

AttributeError: module 'matplotlib.pyplot' has no attribute 'add_artist'

In [53]:
import re
from astropy.coordinates import SkyCoord
from regions import CircleSkyRegion, PixCoord
from astropy import units as u

# Read the region file
file_path = KBSSpath+f"/AbsHosts_nLOS_reg/{field}_AbsHost-nLOS.reg"
with open(file_path, 'r') as file:
    lines = file.readlines()

# Regex to extract information
pattern = re.compile(r'fk5;circle\((.*?),(.*?),(.*?)i\) #color=(.*?)  width=2 text={(.*?)}')

regions = []

for line in lines:
    match = pattern.match(line)
    if match:
        ra, dec, radius, color, text = match.groups()
        coord = SkyCoord(ra, dec, unit=(u.hourangle, u.deg), frame='fk5')
        radius = float(radius) * u.arcsec  # assuming the radius is in arcseconds
        
        # Create a CircleSkyRegion
        region = CircleSkyRegion(center=coord, radius=radius)
        regions.append(region)

# Now `regions` contains all the parsed CircleSkyRegions
# You can manipulate them or plot them using regions library

# Example: Printing region details
for region in regions:
    print(region)

Region: CircleSkyRegion
center: <SkyCoord (FK5: equinox=J2000.000): (ra, dec) in deg
    (15.79699292, 13.27175306)>
radius: 3.0 arcsec
Region: CircleSkyRegion
center: <SkyCoord (FK5: equinox=J2000.000): (ra, dec) in deg
    (15.80051, 13.2711325)>
radius: 5.0 arcsec
Region: CircleSkyRegion
center: <SkyCoord (FK5: equinox=J2000.000): (ra, dec) in deg
    (15.80004083, 13.271705)>
radius: 5.0 arcsec
Region: CircleSkyRegion
center: <SkyCoord (FK5: equinox=J2000.000): (ra, dec) in deg
    (15.80209958, 13.27299472)>
radius: 5.0 arcsec
Region: CircleSkyRegion
center: <SkyCoord (FK5: equinox=J2000.000): (ra, dec) in deg
    (15.79884458, 13.27309694)>
radius: 5.0 arcsec
Region: CircleSkyRegion
center: <SkyCoord (FK5: equinox=J2000.000): (ra, dec) in deg
    (15.79855333, 13.27198778)>
radius: 5.0 arcsec
Region: CircleSkyRegion
center: <SkyCoord (FK5: equinox=J2000.000): (ra, dec) in deg
    (15.79724583, 13.27124722)>
radius: 5.0 arcsec
Region: CircleSkyRegion
center: <SkyCoord (FK5: equino